In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

23/11/15 22:24:21 WARN Utils: Your hostname, MacBook-Air-5.local resolves to a loopback address: 127.0.0.1; using 10.0.0.165 instead (on interface en0)
23/11/15 22:24:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/15 22:24:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)



In [8]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")
df.printSchema()


root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [9]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

query3 = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY date_built
"""

result3 = spark.sql(query3)
result3.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2015|307908.86|
|      2013|299999.39|
|      2014|299073.89|
|      2012|298233.42|
|      2016|296050.24|
|      2010|296800.75|
|      2011| 302141.9|
|      2017|296576.69|
+----------+---------+



In [10]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query4 = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
"""

result4 = spark.sql(query4)
result4.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2015| 288770.3|
|      2013|295962.27|
|      2014|290852.27|
|      2012|293683.19|
|      2016|290555.07|
|      2010|292859.62|
|      2011|291117.47|
|      2017|292676.79|
+----------+---------+



In [11]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query5 = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
"""

result5 = spark.sql(query5)
result5.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2015|297609.97|
|      2013|303676.79|
|      2014|298264.72|
|      2012|307539.97|
|      2016| 293965.1|
|      2010|285010.22|
|      2011|276553.81|
|      2017|280317.58|
+----------+---------+



In [12]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query6 = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
"""

result6 = spark.sql(query6)
result6.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 0.5424392223358154 seconds ---


In [17]:
# 7. Cache the the temporary table home_sales.
query_cached=spark.catalog.cacheTable("home_sales")

23/11/15 22:32:14 WARN CacheManager: Asked to cache already cached data.


In [18]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [20]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

query_uncached = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
"""

# Uncached runtime
start_time_uncached = time.time()
result_uncached = spark.sql(query_uncached)
result_uncached.show()
uncached_runtime = time.time() - start_time_uncached
print("--- Uncached Runtime: %s seconds ---" % uncached_runtime)

# Cached runtime
start_time_cached = time.time()
# Use the cached data
result_cached = spark.sql(query_uncached)
result_cached.show()
cached_runtime = time.time() - start_time_cached
print("--- Cached Runtime: %s seconds ---" % cached_runtime)

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- Uncached Runtime: 0.37235569953918457 seconds ---
+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- Cached Runtime: 0.19146418

In [21]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
parquet_path = "home_sales_partitioned.parquet"

df.write.partitionBy("date_built").parquet(parquet_path, mode="overwrite")

In [23]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet("home_sales_partitioned.parquet")



In [24]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_home_sales")

In [26]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()
query_parquet = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM parquet_home_sales
WHERE price >= 350000
GROUP BY view
"""

result_parquet = spark.sql(query_parquet)
result_parquet.show()

parquet_runtime = time.time() - start_time


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 0.7646791934967041 seconds ---


In [27]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [28]:
# 15. Check if the home_sales is no longer cached
is_cached_after_uncache = spark.catalog.isCached("home_sales")
print("Is home_sales cached after uncaching?", is_cached_after_uncache)


Is home_sales cached after uncaching? False
